In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup as bs
import re
import smtplib
from email.mime.text import MIMEText
import time
from private import privateManager

In [6]:
def no_space(text):
  text1 = re.sub('\nbsp;|&nbsp;|\n|\t|\r|  ', '', text)
  text2 = re.sub('\n\n', '', text1)
  return text2

In [42]:
def check_smu_id(ids):
    res = []
    with open('private/data_smu.txt', 'r') as f:
        datum = f.readline().split(' ')
    for id in ids:
        if id not in datum:
            res.append(id)
    return res

In [55]:
def add_smu_id(ids):
    f = open('private/data_smu.txt', 'r')
    datum = f.readline().split(' ')
    for id in ids:
        if id not in datum:
            datum.append(id)
    datum = ' '.join(datum)
    f.close()
    f = open('private/data_smu.txt', 'w')
    f.write(datum)
    f.close()

In [65]:
def set_all_read(session):
    crawl_smu(session, True)

In [66]:
set_all_read(session)

In [68]:
def crawl_smu(session, set_all_read = False, ignore_cheon = True):
    endpoint = 'https://www.smu.ac.kr/lounge/notice/notice.do'
    soup = bs(session.get(endpoint).text, 'html.parser')
    tableText = soup.find('ul', {'class' : 'board-thumb-wrap'})
    contents = tableText.findAll('dl', {'class' : 'board-thumb-content-wrap'})
    articles = []
    ids = []
    for content in contents:
        tmp = content.find('dd', {'class' : 'board-thumb-content-info'})
        id = no_space(tmp.find('li', {'class' : 'board-thumb-content-number'}).text.strip()).replace('No.', '')
        date = re.findall(r'[0-9]+-[0-9]+-[0-9]+', tmp.find('li', {'class', 'board-thumb-content-date'}).text)[0]
        region = content.find('span', {'class' : 'cmp'}).text
        category = content.find('span', {'class' : 'cate'}).text
        tmp_url = 'https://www.smu.ac.kr/lounge/notice/notice.do?mode=view&articleNo=' + id + '&article.offset=0&articleLimit=10'
        title = no_space([a.text for a in content.find('dt', {'class' : 'board-thumb-content-title'}).findAll('a')][-1])
        article = {
            'region' : region,
            'category' : category,
            'url' : tmp_url,
            'title' : title,
            'date' : date,
            'id' : id
        }
        articles.append(article)
        ids.append(id)
    ids = check_smu_id(ids)
    if set_all_read:
        add_smu_id(ids)
        return
    delList = []
    for (i, article) in enumerate(articles):
        if article['id'] not in ids:
            delList.append(i)
        elif ignore_cheon & article['region'] == '천안':
            delList.append(i)
    for i in delList:
        articles[i] = None
    return articles

In [47]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('https://', adapter)

In [70]:
ids = crawl_smu(session)
ids

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [71]:
def config_mail(contents):
    with open('mailFrame.html', 'r') as f:
        frame = ' '.join(f.readlines())
    for i in range(len(contents)):
        if contents[i] == None:
            continue
        with open('content.html', 'r') as f:
            c = ' '.join(f.readlines())
        c = c.replace('__region__', contents[i]['region'])
        c = c.replace('__url__', contents[i]['url'])
        c = c.replace('__title__', contents[i]['title'])
        c = c.replace('__date__', contents[i]['date'])
        c = c.replace('__category__', contents[i]['category'])
        if i + 1 != len(contents):
            c += '__content__'
        frame = frame.replace('__content__', c)
    return frame

In [72]:
config_mail(ids)

'<div class="content" id="jwxe_main_content" style="text-decoration: none;list-style-type: none;margin: 0;padding: 0;">\n     <div class="common-board  total_notice" id="ko" style="text-decoration: none;list-style-type: none;margin: 0;padding: 0;font-weight: 400;font-size: 17px;color: #333;line-height: 1;max-width: 1200px;position: relative;">\n         <div class="board-name-thumb board-wrap" style="text-decoration: none;list-style-type: none;margin: 0;padding: 0;">\n             <ul class="board-thumb-wrap" style="text-decoration: none;list-style-type: none;margin: 0;padding: 0;">\n                 __content__\n             </ul>\t\n         </div>\n     </div>\t\t\t\n </div>'